# Report on Beach Venues

## Introduction
During summer vacation is common to go to beaches and their seaside venues and enjoy the warm weather. Many famous beaches nowadays have been developing during many years and are in a continuos process of renewing and improving its venues.<br>
Our report aims to evaluate the top beaches of the world and the kind of venues they have in common, helping guide future beach developers/city planners to invest in best venues to increase turism.

## Data Description
Data will be collected from different sources:
* To identify the best beaches in the world, data will be collected from the website TripAdvisor. Anually they make a traveller choice award that rank the best turist attractions around the world. For this report will be used the Travellers' Choice 2021 top beaches in the world and in each region [1].
* To identify the best venues in each beach will be used Foursquare API [2].
* Beaches are going to be grouped by venues similarities, creating different kinds of beaches categories that can be choose to atract tourists.

## Methodology

## Results

## Discussion

## Conclusion

## References
[1] Tripadvisor - Travellers' Choice 2021:
* [World](https://www.tripadvisor.com.br/TravelersChoice-Beaches-cTop-g1)
* [Caribbean](https://www.tripadvisor.com.br/TravelersChoice-Beaches-cTop-g147237)
* [Mexico](https://www.tripadvisor.com.br/TravelersChoice-Beaches-cTop-g150768)
* [United States](https://www.tripadvisor.com.br/TravelersChoice-Beaches-cTop-g191)
* [South America](https://www.tripadvisor.com.br/TravelersChoice-Beaches-cTop-g13)
* [Europe](https://www.tripadvisor.com.br/TravelersChoice-Beaches-cTop-g4)
* [South Pacific](https://www.tripadvisor.com.br/TravelersChoice-Beaches-cTop-g8)
* [Africa](https://www.tripadvisor.com.br/TravelersChoice-Beaches-cTop-g6)
* [Asia](https://www.tripadvisor.com.br/TravelersChoice-Beaches-cTop-g2)

[2] [FourSquare API](https://developer.foursquare.com/)